In [1]:
from matplotlib.colors import ListedColormap
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_moons, make_circles, make_classification
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.lda import LDA
from sklearn.qda import QDA
from sklearn.manifold import TSNE
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Merge, TimeDistributed, LSTM
from keras.layers.core import Dense, Dropout, Activation, Flatten, Reshape
from keras.layers.convolutional import Convolution1D, MaxPooling1D, Convolution2D, MaxPooling2D, ZeroPadding1D, Convolution3D, MaxPooling3D, AveragePooling3D
from keras.optimizers import SGD, Adadelta
from keras.callbacks import EarlyStopping
from keras.layers import Input, merge
from keras.models import Model
from keras import backend as K
from sklearn.cross_validation import train_test_split
import pandas as pd
import numpy as np

C:\Program Files\Anaconda3\lib\site-packages\sklearn\lda.py:4: DeprecationWarning: lda.LDA has been moved to discriminant_analysis.LinearDiscriminantAnalysis in 0.17 and will be removed in 0.19
  "in 0.17 and will be removed in 0.19", DeprecationWarning)
C:\Program Files\Anaconda3\lib\site-packages\sklearn\qda.py:4: DeprecationWarning: qda.QDA has been moved to discriminant_analysis.QuadraticDiscriminantAnalysis in 0.17 and will be removed in 0.19.
  "in 0.17 and will be removed in 0.19.", DeprecationWarning)
Using TensorFlow backend.


In [2]:
data = pd.read_csv('./ex.csv', header = None)
data = data.reindex(np.random.permutation(data.index))
train, test = train_test_split(data, train_size=0.9, random_state=0)
x_train, x_test = train.iloc[:,1:].values, test.iloc[:,1:].values
y_train, y_test = np_utils.to_categorical(train.iloc[:,0].values, num_classes=2), np_utils.to_categorical(test.iloc[:,0].values, num_classes=2)

In [3]:
input_shape = (25600, )
output_shape = 2

In [40]:
# MLP Model
amodel = Sequential()
amodel.add(Reshape(input_shape=input_shape, target_shape=(25600,)))

attention_model = Sequential()
attention_model.add(amodel)
attention_model.add((Activation('softmax', name='Attention_softmax')))
model = Sequential()
model.add(Merge([amodel, attention_model], mode='mul'))




model.add(Dense(500, input_shape=input_shape))
model.add(Activation('tanh'))
model.add(Dense(500))
model.add(Activation('tanh'))
model.add(Dense(500))
model.add(Activation('tanh'))
model.add(Dropout(0.25))
model.add(Dense(output_shape))
model.add(Activation('softmax'))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
merge_2 (Merge)              (None, 25600)             0         
_________________________________________________________________
dense_31 (Dense)             (None, 500)               12800500  
_________________________________________________________________
activation_70 (Activation)   (None, 500)               0         
_________________________________________________________________
dense_32 (Dense)             (None, 500)               250500    
_________________________________________________________________
activation_71 (Activation)   (None, 500)               0         
_________________________________________________________________
dense_33 (Dense)             (None, 500)               250500    
_________________________________________________________________
activation_72 (Activation)   (None, 500)               0         
__________

C:\Program Files\Anaconda3\lib\site-packages\ipykernel\__main__.py:9: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.


In [43]:
# 3D-CNN Model
amodel = Sequential()
amodel.add(Reshape(input_shape=input_shape, target_shape=(8, 8, 8, 50)))


attention_model = Sequential()
attention_model.add(amodel)
attention_model.add((Activation('softmax', name='Attention_softmax')))
model = Sequential()
model.add(Merge([amodel, attention_model], mode='mul'))


model.add(Convolution3D(64, 3, 3, 3, border_mode='valid'))
model.add(Activation('tanh'))
model.add(Convolution3D(64, 3, 3, 3, border_mode='valid'))
model.add(Activation('tanh'))
model.add(Convolution3D(64, 3, 3, 3, border_mode='valid'))
model.add(Activation('tanh'))
model.add(MaxPooling3D(pool_size=(2, 2, 2)))
model.add(Activation('tanh'))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128))
model.add(Activation('tanh'))
model.add(Dropout(0.5))
model.add(Dense(output_shape))
model.add(Activation('softmax'))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
merge_3 (Merge)              (None, 8, 8, 8, 50)       0         
_________________________________________________________________
conv3d_16 (Conv3D)           (None, 6, 6, 6, 64)       86464     
_________________________________________________________________
activation_74 (Activation)   (None, 6, 6, 6, 64)       0         
_________________________________________________________________
conv3d_17 (Conv3D)           (None, 4, 4, 4, 64)       110656    
_________________________________________________________________
activation_75 (Activation)   (None, 4, 4, 4, 64)       0         
_________________________________________________________________
conv3d_18 (Conv3D)           (None, 2, 2, 2, 64)       110656    
_________________________________________________________________
activation_76 (Activation)   (None, 2, 2, 2, 64)       0         
__________

C:\Program Files\Anaconda3\lib\site-packages\ipykernel\__main__.py:10: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
C:\Program Files\Anaconda3\lib\site-packages\ipykernel\__main__.py:13: UserWarning: Update your `Conv3D` call to the Keras 2 API: `Conv3D(64, (3, 3, 3), padding="valid")`
C:\Program Files\Anaconda3\lib\site-packages\ipykernel\__main__.py:15: UserWarning: Update your `Conv3D` call to the Keras 2 API: `Conv3D(64, (3, 3, 3), padding="valid")`
C:\Program Files\Anaconda3\lib\site-packages\ipykernel\__main__.py:17: UserWarning: Update your `Conv3D` call to the Keras 2 API: `Conv3D(64, (3, 3, 3), padding="valid")`


In [37]:
# 2D-CNN Model
amodel = Sequential()
amodel.add(Reshape(input_shape=input_shape, target_shape=(64, 8, 50)))

attention_model = Sequential()
attention_model.add(amodel)
attention_model.add((Activation('softmax', name='Attention_softmax')))
model = Sequential()
model.add(Merge([amodel, attention_model], mode='mul'))

model.add(Convolution2D(64, 3, 3, border_mode='valid'))
model.add(Activation('tanh'))
model.add(Convolution2D(64, 3, 3, border_mode='valid'))
model.add(Activation('tanh'))
model.add(Convolution2D(64, 3, 3, border_mode='valid'))
model.add(Activation('tanh'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Activation('tanh'))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128))
model.add(Activation('tanh'))
model.add(Dropout(0.5))
model.add(Dense(output_shape))
model.add(Activation('softmax'))

model.summary()

C:\Program Files\Anaconda3\lib\site-packages\ipykernel\__main__.py:9: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
C:\Program Files\Anaconda3\lib\site-packages\ipykernel\__main__.py:14: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), padding="valid")`
C:\Program Files\Anaconda3\lib\site-packages\ipykernel\__main__.py:16: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), padding="valid")`
C:\Program Files\Anaconda3\lib\site-packages\ipykernel\__main__.py:18: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), padding="valid")`


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
merge_1 (Merge)              (None, 64, 8, 50)         0         
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 62, 6, 64)         28864     
_________________________________________________________________
activation_64 (Activation)   (None, 62, 6, 64)         0         
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 60, 4, 64)         36928     
_________________________________________________________________
activation_65 (Activation)   (None, 60, 4, 64)         0         
_________________________________________________________________
conv2d_15 (Conv2D)           (None, 58, 2, 64)         36928     
_________________________________________________________________
activation_66 (Activation)   (None, 58, 2, 64)         0         
__________

In [4]:
# Model (3D-CNN + LSTM)
# Model Start
amodel = Sequential()
amodel.add(Reshape(input_shape=input_shape, target_shape=(50, 8, 8, 8, 1)))

# Attention
attention_model = Sequential()
attention_model.add(amodel)
attention_model.add((Activation('softmax', name='Attention_softmax')))
model = Sequential()
model.add(Merge([amodel, attention_model], mode='mul'))

# 3D-CNN
model.add(TimeDistributed(Convolution3D(64, 3, 3, 3, border_mode='valid', activation='tanh')))
model.add(TimeDistributed(Convolution3D(128, 3, 3, 3, border_mode='valid', activation='tanh')))
model.add(TimeDistributed(Convolution3D(256, 3, 3, 3, border_mode='valid', activation='tanh')))
model.add(TimeDistributed(MaxPooling3D(pool_size=(2, 2, 2))))
model.add(TimeDistributed(Activation('tanh')))
model.add(TimeDistributed(Dropout(0.25)))
model.add(TimeDistributed(Flatten()))


# LSTM
model.add(LSTM(50, return_sequences=False))
model.add(Dense(output_shape))

model.summary()

C:\Program Files\Anaconda3\lib\site-packages\ipykernel\__main__.py:11: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
C:\Program Files\Anaconda3\lib\site-packages\ipykernel\__main__.py:14: UserWarning: Update your `Conv3D` call to the Keras 2 API: `Conv3D(64, (3, 3, 3), padding="valid", activation="tanh")`
C:\Program Files\Anaconda3\lib\site-packages\ipykernel\__main__.py:15: UserWarning: Update your `Conv3D` call to the Keras 2 API: `Conv3D(128, (3, 3, 3), padding="valid", activation="tanh")`
C:\Program Files\Anaconda3\lib\site-packages\ipykernel\__main__.py:16: UserWarning: Update your `Conv3D` call to the Keras 2 API: `Conv3D(256, (3, 3, 3), padding="valid", activation="tanh")`


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
merge_1 (Merge)              (None, 50, 8, 8, 8, 1)    0         
_________________________________________________________________
time_distributed_1 (TimeDist (None, 50, 6, 6, 6, 64)   1792      
_________________________________________________________________
time_distributed_2 (TimeDist (None, 50, 4, 4, 4, 128)  221312    
_________________________________________________________________
time_distributed_3 (TimeDist (None, 50, 2, 2, 2, 256)  884992    
_________________________________________________________________
time_distributed_4 (TimeDist (None, 50, 1, 1, 1, 256)  0         
_________________________________________________________________
time_distributed_5 (TimeDist (None, 50, 1, 1, 1, 256)  0         
_________________________________________________________________
time_distributed_6 (TimeDist (None, 50, 1, 1, 1, 256)  0         
__________

In [5]:
model.compile(optimizer='adadelta',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [6]:
history = model.fit(x_train, y_train,
                    batch_size=100,
                    epochs=10,
                    verbose=1,
                    validation_data=(x_test, y_test))

Train on 2223 samples, validate on 247 samples
Epoch 1/10
 400/2223 [====>.........................] - ETA: 5013s - loss: 5.8153 - acc: 0.5225

KeyboardInterrupt: 